<a href="https://colab.research.google.com/github/andongl1997/CIS550-Final-Project/blob/main/CIS550_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part 0 - Settings

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/Shareddrives/cis550_project/Covid_outline

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/cis550_project/Covid_outline


In [ ]:
import numpy as np
import pandas as pd
!pip3 install pandasql
import pandasql as ps

#Part 1 - Data Clean

##Map page

In [ ]:
df = pd.read_csv('covid-19-all.csv')
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
0,NaN,NaN,NaN,NaN,51526.0,41727.0,2191.0,2021-01-01
1,NaN,NaN,NaN,NaN,58316.0,33634.0,1181.0,2021-01-01
2,NaN,NaN,NaN,NaN,99897.0,67395.0,2762.0,2021-01-01
3,NaN,NaN,NaN,NaN,8117.0,7463.0,84.0,2021-01-01
4,NaN,NaN,NaN,NaN,17568.0,11146.0,405.0,2021-01-01


In [ ]:
# Check the Null values 
df.isnull().sum()

Country/Region    171061
Province/State    223208
Latitude          171062
Longitude         171062
Confirmed             19
Recovered            386
Deaths               432
Date                   0
dtype: int64

In [ ]:
df = df.rename(columns={'Country/Region':'Country_Region',
          'Province/State':'Province_State'})
df.head()

,Country_Region,Province_State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
0,NaN,NaN,NaN,NaN,51526.0,41727.0,2191.0,2021-01-01
1,NaN,NaN,NaN,NaN,58316.0,33634.0,1181.0,2021-01-01
2,NaN,NaN,NaN,NaN,99897.0,67395.0,2762.0,2021-01-01
3,NaN,NaN,NaN,NaN,8117.0,7463.0,84.0,2021-01-01
4,NaN,NaN,NaN,NaN,17568.0,11146.0,405.0,2021-01-01


In [ ]:
# df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
#Since the country is the prime key, we don't expect null values here \
query = """ 
Select Country_Region,Province_State,Confirmed,Recovered,Deaths,Date
From df 
where Country_Region is not null """
map_df = ps.sqldf(query)

In [ ]:
#Fill nulls with 0 in Confirmed,Recovered,Deaths cases 
map_df['Confirmed'].fillna(0,inplace = True)
map_df['Recovered'].fillna(0,inplace = True)
map_df['Deaths'].fillna(0,inplace = True)
map_df

,Country_Region,Province_State,Confirmed,Recovered,Deaths,Date
0,China,Anhui,1.0,0.0,0.0,2020-01-22
1,China,Beijing,14.0,0.0,0.0,2020-01-22
2,China,Chongqing,6.0,0.0,0.0,2020-01-22
3,China,Fujian,1.0,0.0,0.0,2020-01-22
4,China,Gansu,0.0,0.0,0.0,2020-01-22
...,...,...,...,...,...,...
1070886,Vietnam,None,1465.0,1325.0,35.0,2020-12-31
1070887,West Bank and Gaza,None,138004.0,117183.0,1400.0,2020-12-31
1070888,Yemen,None,2099.0,1394.0,610.0,2020-12-31
1070889,Zambia,None,20725.0,18660.0,388.0,2020-12-31


In [ ]:
confirm_case = map_df[['Country_Region','Province_State','Confirmed','Date']]
recover_case = map_df[['Country_Region','Province_State','Recovered','Date']]
deaths_case = map_df[['Country_Region','Province_State','Deaths','Date']]

In [ ]:
confirm_case

,Country_Region,Province_State,Confirmed,Date
0,China,Anhui,1.0,2020-01-22
1,China,Beijing,14.0,2020-01-22
2,China,Chongqing,6.0,2020-01-22
3,China,Fujian,1.0,2020-01-22
4,China,Gansu,0.0,2020-01-22
...,...,...,...,...
1070886,Vietnam,None,1465.0,2020-12-31
1070887,West Bank and Gaza,None,138004.0,2020-12-31
1070888,Yemen,None,2099.0,2020-12-31
1070889,Zambia,None,20725.0,2020-12-31


In [ ]:
confirm_case.to_csv('Confirm_Case.csv')
recover_case.to_csv('Recover_Case.csv')
deaths_case.to_csv('Deaths_Case.csv')


In [ ]:
confirm_case

,Country_Region,Province_State,Confirmed,Date
0,China,Anhui,1.0,2020-01-22
1,China,Beijing,14.0,2020-01-22
2,China,Chongqing,6.0,2020-01-22
3,China,Fujian,1.0,2020-01-22
4,China,Gansu,0.0,2020-01-22
...,...,...,...,...
1070886,Vietnam,None,1465.0,2020-12-31
1070887,West Bank and Gaza,None,138004.0,2020-12-31
1070888,Yemen,None,2099.0,2020-12-31
1070889,Zambia,None,20725.0,2020-12-31


In [ ]:
#Given the rank of confirmed,death recovered for each country based on the number 
query = '''
With temp1 as (
Select Country_Region, sum(Recovered) as Recovered, 1 as rank
from recover_case
group by Country_Region
order by Recovered desc),

temp_recover_rank as(
select b.Country_Region,sum(a.rank) as recover_rank
from temp1 a,temp1 b 
where a.Recovered>=b.Recovered 
group by b.Country_Region
order by recover_rank asc),

temp2 as (
Select Country_Region, sum(Deaths) as Deaths, 1 as death_rank
from deaths_case
group by Country_Region
order by Deaths desc),

temp_death_rank as(
select d.Country_Region,sum(c.death_rank) as death_rank
from temp2 c,temp2 d 
where c.Deaths>=d.Deaths 
group by d.Country_Region
order by death_rank asc),


temp3 as(
Select Country_Region, sum(Confirmed) as Confirmed, 1 as confirmed_rank
from confirm_case
group by Country_Region
order by Confirmed desc),

temp_confirmed_rank as(
select f.Country_Region,sum(e.confirmed_rank) as confirmed_rank
from temp3 e,temp3 f 
where e.Confirmed>=f.Confirmed 
group by f.Country_Region
order by confirmed_rank asc),

final_temp as (
select * from temp_confirmed_rank
join temp_recover_rank using(Country_Region)
join temp_death_rank using(Country_Region))

select * 
from final_temp 
where Country_Region = 'US';

'''


ps.sqldf(query)

,Country_Region,confirmed_rank,recover_rank,death_rank
0,US,1,3,1


##Food quantity


In [ ]:
df = pd.read_csv('Food_Supply_Quantity_kg_Data.csv')
df.head()

,Country,Alcoholic Beverages,Animal fats,Animal Products,"Aquatic Products, Other",Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,Milk - Excluding Butter,Miscellaneous,Offals,Oilcrops,Pulses,Spices,Starchy Roots,Stimulants,Sugar & Sweeteners,Sugar Crops,Treenuts,Vegetable Oils,Vegetables,Vegetal Products,Obesity,Undernourished,Confirmed,Deaths,Recovered,Active,Population,Unit (all except Population)
0,Afghanistan,0.0014,0.1973,9.4341,0.0,24.8097,0.2099,0.0350,5.3495,1.2020,7.5828,0.0728,0.2057,0.0700,0.2953,0.0574,0.8802,0.3078,1.3489,0.0,0.0770,0.5345,6.7642,40.5645,4.5,29.8,0.142134,0.006186,0.123374,0.012574,38928000.0,%
1,Albania,1.6719,0.1357,18.7684,0.0,5.7817,0.5815,0.2126,6.7861,1.8845,15.7213,0.1123,0.2324,0.9377,0.2380,0.0008,1.8096,0.1055,1.5367,0.0,0.1515,0.3261,11.7753,31.2304,22.3,6.2,2.967301,0.050951,1.792636,1.123714,2838000.0,%
2,Algeria,0.2711,0.0282,9.6334,0.0,13.6816,0.5277,0.2416,6.3801,1.1305,7.6189,0.1671,0.0870,0.3493,0.4783,0.0557,4.1340,0.2216,1.8342,0.0,0.1152,1.0310,11.6484,40.3651,26.6,3.9,0.244897,0.006558,0.167572,0.070767,44357000.0,%
3,Angola,5.8087,0.0560,4.9278,0.0,9.1085,0.0587,1.7707,6.0005,2.0571,0.8311,0.1165,0.1550,0.4186,0.6507,0.0009,18.1102,0.0508,1.8495,0.0,0.0061,0.6463,2.3041,45.0722,6.8,25,0.061687,0.001461,0.056808,0.003419,32522000.0,%
4,Antigua and Barbuda,3.5764,0.0087,16.6613,0.0,5.9960,0.2274,4.1489,10.7451,5.6888,6.3663,0.7139,0.2219,0.2172,0.1840,0.1524,1.4522,0.1564,3.8749,0.0,0.0253,0.8102,5.4495,33.3233,19.1,NaN,0.293878,0.007143,0.190816,0.095918,98000.0,%


In [ ]:
#Delete meaningless features
df = df.drop(['Aquatic Products, Other','Miscellaneous','Offals','Oilcrops','Pulses','Spices','Stimulants','Sugar Crops','Treenuts','Active','Unit (all except Population)'],axis = 1)

In [ ]:
#Fill the null values with 0
df.fillna(0,inplace = True)

In [ ]:
df.columns

Index(['Country', 'Alcoholic Beverages', 'Animal fats', 'Animal Products',
       'Cereals - Excluding Beer', 'Fish, Seafood', 'Fruits - Excluding Wine',
       'Meat', 'Milk - Excluding Butter', 'Starchy Roots',
       'Sugar & Sweeteners', 'Vegetable Oils', 'Vegetables',
       'Vegetal Products', 'Obesity', 'Undernourished', 'Confirmed', 'Deaths',
       'Recovered', 'Population'],
      dtype='object')

In [ ]:
#Divide the dataset into two parts for the future application
df_food = df[['Country', 'Alcoholic Beverages', 'Animal fats', 'Animal Products',
       'Cereals - Excluding Beer','Eggs', 'Fish, Seafood', 'Fruits - Excluding Wine',
       'Meat', 'Milk - Excluding Butter', 'Starchy Roots',
       'Sugar & Sweeteners', 'Vegetable Oils', 'Vegetables',
       'Vegetal Products', 'Obesity', 'Undernourished']]
df_food_case = df[['Country','Confirmed', 'Deaths','Recovered', 'Population']]

In [ ]:
#Replace the names of some columns for sql queries
df_food = df_food.rename(columns={'Alcoholic Beverages':'Alcoholic_Beverages',
            'Animal fats':'Animal_fats',
            'Animal Products':'Animal_Products',
            'Cereals - Excluding Beer':'Cereals',
            'Fish, Seafood' : 'Fish_Seafood',
            'Fruits - Excluding Wine':'Fruits',
            'Milk - Excluding Butter':'Milk',
            'Starchy Roots':'Starchy_Roots',
            'Sugar & Sweeteners':'Sugar',
            'Vegetable Oils': 'Vegetable_Oils',
            'Vegetal Products' :'Vegetal_Products'})

In [ ]:
df_food.head()  

,Country,Alcoholic_Beverages,Animal_fats,Animal_Products,Cereals,Fish_Seafood,Fruits,Meat,Milk,Starchy_Roots,Sugar,Vegetable_Oils,Vegetables,Vegetal_Products,Obesity,Undernourished
0,Afghanistan,0.0014,0.1973,9.4341,24.8097,0.0350,5.3495,1.2020,7.5828,0.8802,1.3489,0.5345,6.7642,40.5645,4.5,29.8
1,Albania,1.6719,0.1357,18.7684,5.7817,0.2126,6.7861,1.8845,15.7213,1.8096,1.5367,0.3261,11.7753,31.2304,22.3,6.2
2,Algeria,0.2711,0.0282,9.6334,13.6816,0.2416,6.3801,1.1305,7.6189,4.1340,1.8342,1.0310,11.6484,40.3651,26.6,3.9
3,Angola,5.8087,0.0560,4.9278,9.1085,1.7707,6.0005,2.0571,0.8311,18.1102,1.8495,0.6463,2.3041,45.0722,6.8,25
4,Antigua and Barbuda,3.5764,0.0087,16.6613,5.9960,4.1489,10.7451,5.6888,6.3663,1.4522,3.8749,0.8102,5.4495,33.3233,19.1,0


In [ ]:
df_food_case.head()

,Country,Confirmed,Deaths,Recovered,Population
0,Afghanistan,0.142134,0.006186,0.123374,38928000.0
1,Albania,2.967301,0.050951,1.792636,2838000.0
2,Algeria,0.244897,0.006558,0.167572,44357000.0
3,Angola,0.061687,0.001461,0.056808,32522000.0
4,Antigua and Barbuda,0.293878,0.007143,0.190816,98000.0


##Covid Groups

In [ ]:
df = pd.read_csv('COVID-19_Case_Surveillance_Public_Use_Data.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#Cleat the 'sex' column
################################
############Full RAM###########
# query = '''
# Select * from df 
# where sex != 'Other' and sex != 'Missing' and sex != 'Unknown'
# '''
# df = ps.sqldf(query)
# df[['sex']].value_counts()
################################
# Clear the sex column
df = df[(df['sex'] != 'Unknown') & (df['sex']!= 'Missing')  & (df['sex']!= 'Other')]
df.reset_index(inplace= True,drop= True)

# Clear the age_group column
df = df[(df['age_group'] != 'Unknown')]
df.reset_index(inplace= True,drop= True)

# Clear the Race and ethnicity (combined) column
df = df[(df['race_ethnicity_combined'] != 'Unknown') & (df['race_ethnicity_combined'] != 'Missing')]
df.reset_index(inplace= True,drop= True)

# Clear the hosp_yn column
df = df[(df['hosp_yn'] != 'Unknown') & (df['hosp_yn'] != 'Missing')]
df.reset_index(inplace= True,drop= True)
# Clear the icu_yn column
df = df[(df['icu_yn'] != 'Unknown') & (df['icu_yn'] != 'Missing')] 
df.reset_index(inplace= True,drop= True)
# Clear the death_yn column 
df = df[(df['death_yn'] != 'Unknown') & (df['death_yn'] != 'Missing')]
df.reset_index(inplace= True,drop= True)


In [ ]:
#Select some meaningful columns
df.columns

Index(['cdc_case_earliest_dt ', 'cdc_report_dt', 'pos_spec_dt', 'onset_dt',
       'current_status', 'sex', 'age_group', 'race_ethnicity_combined',
       'hosp_yn', 'icu_yn', 'death_yn', 'medcond_yn'],
      dtype='object')

In [ ]:
df = df[['sex', 'age_group', 'race_ethnicity_combined',
       'hosp_yn', 'icu_yn', 'death_yn']]
df

,sex,age_group,race_ethnicity_combined,hosp_yn,icu_yn,death_yn
0,Female,0 - 9 Years,"White, Non-Hispanic",No,No,No
1,Female,0 - 9 Years,"White, Non-Hispanic",No,No,No
2,Male,0 - 9 Years,Hispanic/Latino,No,No,No
3,Male,0 - 9 Years,Hispanic/Latino,No,No,No
4,Male,0 - 9 Years,Hispanic/Latino,No,No,No
...,...,...,...,...,...,...
824452,Female,30 - 39 Years,"Multiple/Other, Non-Hispanic",No,No,No
824453,Female,30 - 39 Years,"Multiple/Other, Non-Hispanic",No,No,No
824454,Female,30 - 39 Years,"Native Hawaiian/Other Pacific Islander, Non-Hi...",No,No,No
824455,Female,30 - 39 Years,"Native Hawaiian/Other Pacific Islander, Non-Hi...",No,No,No
